In [1]:
import json
import pandas as pd
from datasets import Dataset
from bs4 import BeautifulSoup
from tqdm import tqdm

In [8]:
data = []
with open('hf-datasets/raw-datasets/data.gov.my.jsonl') as fopen:
    for l in tqdm(fopen):
        data.append(json.loads(l)['text'])

10955it [00:02, 4482.57it/s]


In [9]:
data[0]

'Data ini menerangkan tentang PERUNTUKAN DAN BELANJA JALAN LUAR BANDAR (JALB) sehingga 30 Jun 2018.\nJumlah Dilihat : 37\nData ini menerangkan tentang PERUNTUKAN DAN BELANJA JALAN LUAR BANDAR (JALB)...\nDownloads : 8\n,JALAN LUAR BANDAR (JALB),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4\n0,,,,,\n1,Negeri,2018,,,\n2,,Peruntukan (RM),Belanja (RM),Panjang Jalan (km),Penerima Faedah\n3,Johor,48800000,10711169.91,3.3,2000\n4,Kedah,22700000,3774400,0,0\n5,Kelantan,88991683.13,22771127.009999998,0,0\n6,Melaka,14500000,494593.94,0,0\n7,N. Sembilan,18000000,661570.54,0,0\n8,P. Pinang,5000100,14373.6,0,0\n9,Pahang,112658316.87,10272525.4,0,0\n10,Perak,34600000,6403.65,0,0\n11,Perlis,5200000,0,0,0\n12,Selangor,100000,42983.85,0,0\n13,Terengganu,47800000,3976352.53,0,0\n14,Sabah,253500000,135292129.03,9,3000\n15,Sarawak,129900000,27907837.62,9.6,4500\n16,Jumlah,781750100,215925467.08,21.9,9500\n17,,,,,\n18,Nota : Data sehingga 30 Jun 2018,,,,\n19,"Sumber : Bahagian Prasarana, KPLB",,,,\n'

In [10]:
len(data)

10955

In [11]:
data_dict = {
    'text': data
}

In [12]:
dataset = Dataset.from_dict(data_dict)

In [13]:
dataset

Dataset({
    features: ['text'],
    num_rows: 10955
})

In [14]:
dataset.save_to_disk("hf-datasets/raw-datasets/data.gov.my")

Saving the dataset (0/3 shards):   0%|          | 0/10955 [00:00<?, ? examples/s]

In [15]:
!du -hs hf-datasets/raw-datasets/data.gov.my

980M	hf-datasets/raw-datasets/data.gov.my


### 2. Run below command

make sure directory for the script is correct before running the command

- path -  your dataset to dedupe path
- name - relevant only when u use dataset directly from huggingface
- output - output directory of the dedupe dataset
- column - column of the text we are going to remove duplicate
- threshold - jaccard similarity threshold
- local - (local dataset or directly from huggingface) remove if dataset is not local

In [16]:
command = f"python3 -m text_dedup.minhash \
  --path hf-datasets/raw-datasets/data.gov.my \
  --split train \
  --cache_dir ./cache \
  --output hf-datasets/dedupe-datasets/data.gov.my \
  --column text \
  --batch_size 10000 \
  --threshold 0.95 \
  --min_length 1 \
  --local"

command

'python3 -m text_dedup.minhash   --path hf-datasets/raw-datasets/data.gov.my   --split train   --cache_dir ./cache   --output hf-datasets/dedupe-datasets/data.gov.my   --column text   --batch_size 10000   --threshold 0.95   --min_length 1   --local'

In [18]:
# import subprocess
# subprocess.run(command, shell=True)

The script produced the following

- duplicates.csv : containing duplicate pairs indices (document index)
- duplicate_cluster.csv: the text of duplicate pairs for view
- the deduped dataset in huggingface dataset type (from output argument)
- dedup dataset in jsonl format

In [19]:
!wc -l hf-datasets/dedupe-datasets/data.gov.my.jsonl

10897 hf-datasets/dedupe-datasets/data.gov.my.jsonl


In [20]:
!ls -lh hf-datasets/dedupe-datasets/data.gov.my.jsonl

-rw-r--r-- 1 ubuntu ubuntu 994M Aug 18 03:06 hf-datasets/dedupe-datasets/data.gov.my.jsonl


In [21]:
!rm -rf hf-datasets/dedupe-datasets/data.gov.my hf-datasets/raw-datasets/data.gov.my